In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# パラメタ
learning_rate = 0.001
training_epochs = 200000
batch_size = 128
display_step = 10

# ネットワークパラメタ
n_input = 784
n_classes = 10
dropout = 0.75

x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [3]:
# 畳み込み処理
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding="SAME")
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

# プーリング処理
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding="SAME")

# モデル定義
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # 畳み込み1
    conv1 = conv2d(x, weights["wc1"], biases["bc1"])
    conv1 = maxpool2d(conv1, k=2)
    
    # 畳み込み2
    conv2 = conv2d(conv1, weights["wc2"], biases["bc2"])
    conv2 = maxpool2d(conv2, k=2)
    
    # 全結合
    fc1 = tf.reshape(conv2, [-1, weights["wd1"].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights["wd1"]), biases["bd1"])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout) # dropout を適応
    
    out = tf.add(tf.matmul(fc1, weights["out"]), biases["out"])
    return out

In [4]:
# 重み、バイアス管理用の変数定義
weights = {
    "wc1": tf.Variable(tf.random_normal([5, 5,  1, 32])),
    "wc2": tf.Variable(tf.random_normal([5, 5, 32, 64])),
    "wd1": tf.Variable(tf.random_normal([7*7*64, 1024])),
    "out": tf.Variable(tf.random_normal([1024, n_classes]))
}
biases = {
    "bc1": tf.Variable(tf.random_normal([32])),
    "bc2": tf.Variable(tf.random_normal([64])),
    "bd1": tf.Variable(tf.random_normal([1024])),
    "out": tf.Variable(tf.random_normal([n_classes]))
}

# モデル
pred = conv_net(x, weights, biases, keep_prob)

# ロス関数
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
# オプティマイザ
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 評価用モデル
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 初期化
init = tf.global_variables_initializer()

In [5]:
# 実行
with tf.Session() as sess:
    sess.run(init)
    
    step = 1
    while step * batch_size < training_epochs:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
    
        if (step+1) % display_step == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            print "Epoch:", str(step*batch_size), "Loss=", loss, "Acc=", acc
        
        step += 1

    print "Optimization Finish"
    print "Testing Accuracy:", \
    sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                  y: mnist.test.labels[:256],
                                  keep_prob: 1.})

Epoch: 1152 Loss= 13372.3 Acc= 0.414062
Epoch: 2432 Loss= 6249.95 Acc= 0.617188
Epoch: 3712 Loss= 6529.12 Acc= 0.664062
Epoch: 4992 Loss= 2278.86 Acc= 0.84375
Epoch: 6272 Loss= 4129.24 Acc= 0.8125
Epoch: 7552 Loss= 4896.6 Acc= 0.734375
Epoch: 8832 Loss= 3236.54 Acc= 0.851562
Epoch: 10112 Loss= 2533.47 Acc= 0.882812
Epoch: 11392 Loss= 1509.79 Acc= 0.90625
Epoch: 12672 Loss= 2506.21 Acc= 0.828125
Epoch: 13952 Loss= 406.333 Acc= 0.960938
Epoch: 15232 Loss= 2908.74 Acc= 0.820312
Epoch: 16512 Loss= 1091.98 Acc= 0.898438
Epoch: 17792 Loss= 1875.44 Acc= 0.875
Epoch: 19072 Loss= 1936.71 Acc= 0.914062
Epoch: 20352 Loss= 1651.09 Acc= 0.90625
Epoch: 21632 Loss= 2611.55 Acc= 0.898438
Epoch: 22912 Loss= 1250.94 Acc= 0.9375
Epoch: 24192 Loss= 1041.02 Acc= 0.921875
Epoch: 25472 Loss= 645.373 Acc= 0.929688
Epoch: 26752 Loss= 2792.39 Acc= 0.835938
Epoch: 28032 Loss= 538.389 Acc= 0.945312
Epoch: 29312 Loss= 609.911 Acc= 0.953125
Epoch: 30592 Loss= 390.868 Acc= 0.96875
Epoch: 31872 Loss= 1067.65 Acc= 0.9